<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#sp나-통계적인-숫자-이하의-종-떨굴-필요-x" data-toc-modified-id="sp나-통계적인-숫자-이하의-종-떨굴-필요-x-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>sp나 통계적인 숫자 이하의 종 떨굴 필요 x</a></span></li></ul></div>

# sp나 통계적인 숫자 이하의 종 떨굴 필요 x
-> 어차피 피쳐로 들어가는 종들만 골라서 진행하는 작업이기 때문

In [27]:
import pandas as pd
from tqdm import tqdm 
from haversine import haversine

In [28]:
df = pd.read_csv('../../20220209_07_21_무당벌레관측.csv')
df2 = pd.read_csv('../DOA_bipunctata/220220_DOA_bi(최종,minmax).csv')

In [29]:
df2 = df2[df2.Species=='Adalia bipunctata'].reset_index(drop=True)

In [30]:
df = df[df.public_positional_accuracy<=1000].reset_index(drop=True)

In [31]:
df = df[(df.Latitude!=0.0)&(df.Longitude!=0.0)].reset_index(drop=True)

In [32]:
df.Latitude = df.Latitude.astype(float)
df.Longitude = df.Longitude.astype(float)

df2.Latitude = df2.Latitude.astype(float)
df2.Longitude = df2.Longitude.astype(float)

In [33]:
df_gps = []
for i in tqdm(range(len(df))):
    df_gps.append( (df.Latitude[i], df.Longitude[i]) )
    
    
df2_gps = []
for i in tqdm(range(len(df2))):
    try:
        df2_gps.append( (df2.Latitude[i], df2.Longitude[i]) )
    except:
        print(i)

100%|███████████████████████████████████████████████████████████████████████████| 1438/1438 [00:00<00:00, 72029.73it/s]


In [34]:
df['gps'] = df_gps
df2['gps'] = df2_gps

In [35]:
df2=df2[['Species','gps','Year']]

In [36]:
df_dict = {'year' + str(i): df.loc[df["Year"]==i].sort_values(by="Species").reset_index(drop=True) 
           for i in range(2007,2022)}  

In [37]:
# bipunctata (minmax) PTYPE
# ['Lanius_borealis_18', 'Harmonia_axyridis_18',
#        'Cinclus_mexicanus_18', 'Coccinella_septempunctata_18',
#        'Cycloneda_polita_18', 'Eremophila_alpestris_18',
#        'Coelophora_maculata_18', 'Hippodamia_convergens_18',
#        'Hippodamia_tredecimpunctata_18', 'Anatis_mali_18',
#        'Lanius_ludovicianus_18', 'Hippodamia_variegata_18',
#        'Cycloneda_munda_18', 'Mulsantina_picta_18',
#        'Phainopepla_nitens_18', 'Anatis_lecontei_18',
#        'Centronyx_henslowii_18', 'Calvia_quatuordecimguttata_18',
#        'Olla_vnigrum_18', 'Coccinella_novemnotata_18']

In [38]:
# 언더바 지우기
bass = ['Lanius borealis', 'Harmonia axyridis',
       'Cinclus mexicanus', 'Coccinella septempunctata',
       'Cycloneda polita', 'Eremophila alpestris',
       'Coelophora maculata', 'Hippodamia convergens',
       'Hippodamia tredecimpunctata', 'Anatis mali',
       'Lanius ludovicianus', 'Hippodamia variegata',
       'Cycloneda munda', 'Mulsantina picta',
       'Phainopepla nitens', 'Anatis lecontei',
       'Centronyx henslowii', 'Calvia quatuordecimguttata',
       'Olla v-nigrum', 'Coccinella novemnotata']


PTYPE = []

In [39]:
# 무당벌레만 뽑기
for i in bass:
    if i in df.Species.unique():
        PTYPE.append(i)

In [40]:
# 데이터 프레임에 변수명 생성 (종명 + 년도)
for i in PTYPE:
    for j in range(2007,2022):
        globals()['{}_{}df'.format(i.split(" ")[1],j)] = df_dict['year' + str(j)].loc[df_dict['year' + str(j)].Species==i].reset_index(drop=True)

In [41]:
# 변수명 생성 (종명 + 년도)
for i in range(2007,2022):
    for j in range(len(df2)):
        for k in PTYPE:
            globals()['{}_{}_{}'.format(k.split(" ")[1],i,j)] = 0   

In [42]:
for i in range(2007,2022):
    for j in PTYPE:
        globals()['{}list_{}'.format(j.split(" ")[1],i)] = []

In [43]:
for i in tqdm(range(len(df2.gps))):
    for j in range(2007,2022):
        for name in PTYPE:
            for k in range(len(globals()['{}_{}df'.format(name.split(" ")[1],j)])):
                if haversine(globals()['{}_{}df'.format(name.split(" ")[1],j)].gps[k], df2.gps[i], unit='km') <= 18:
                    globals()['{}_{}_{}'.format(name.split(" ")[1],j,i)] += 1
                    

100%|██████████████████████████████████████████████████████████████████████████████| 1438/1438 [59:40<00:00,  2.49s/it]


In [44]:
for name in PTYPE:
    for j in range(2007,2022):
        for k in range(len(df2.gps)):
            globals()['{}list_{}'.format(name.split(" ")[1],j)].append(globals()['{}_{}_{}'.format(name.split(" ")[1],j,k)])

In [45]:
for name in PTYPE:
    for j in range(2007,2022):
        df2[name.split(" ")[1]+"_"+str(j)+"_18km"] = globals()['{}list_{}'.format(name.split(" ")[1],j)]

In [46]:
df2.describe()

,Year,axyridis_2007_18km,axyridis_2008_18km,axyridis_2009_18km,axyridis_2010_18km,axyridis_2011_18km,axyridis_2012_18km,axyridis_2013_18km,axyridis_2014_18km,axyridis_2015_18km,...,novemnotata_2012_18km,novemnotata_2013_18km,novemnotata_2014_18km,novemnotata_2015_18km,novemnotata_2016_18km,novemnotata_2017_18km,novemnotata_2018_18km,novemnotata_2019_18km,novemnotata_2020_18km,novemnotata_2021_18km
count,1438.000000,1438.000000,1438.000000,1438.000000,1438.000000,1438.000000,1438.000000,1438.000000,1438.000000,1438.000000,...,1438.000000,1438.000000,1438.000000,1438.000000,1438.000000,1438.000000,1438.000000,1438.000000,1438.000000,1438.000000
mean,2017.327538,0.358136,0.549374,0.715577,0.774687,1.078581,1.502086,1.435327,2.659249,3.630042,...,0.093185,0.052156,0.064673,0.022253,0.008345,0.114743,0.067455,0.085535,0.032684,0.170376
std,3.925784,1.569667,1.088305,1.462277,1.886294,1.760567,2.621507,3.495846,4.239608,6.603239,...,0.539022,0.454689,0.287752,0.199666,0.111608,0.594610,0.522598,0.342409,0.181741,0.557893
min,2007.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2015.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2019.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2021.000000,0.000000,1.000000,1.000000,0.000000,2.000000,1.000000,1.000000,4.000000,4.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2021.000000,16.000000,9.000000,13.000000,18.000000,10.000000,11.000000,42.000000,21.000000,25.000000,...,6.000000,11.000000,3.000000,4.000000,2.000000,6.000000,5.000000,3.000000,2.000000,10.000000


In [48]:
df2.to_csv('./20220226_DOA_bipunctata실제테스트_무당벌레(3종).csv')